In [4]:
%load_ext autoreload
%autoreload 2

import torch
import pprint
from pathlib import Path

In [5]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset
from lerobot.policies.factory import make_policy
from lerobot.policies.smolvla.configuration_smolvla import SmolVLAConfig

c:\Users\19142\.conda\envs\my_robotics\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from lerobot.policies.smolvla.modeling_smolvla import SmolVLAPolicy


In [18]:
repo_id = "lerobot/svla_so100_pickplace"

dataset = LeRobotDataset(repo_id, episodes=[0])

print(f"Dataset loaded: {len(dataset)} frames")

pprint.pprint(dataset.meta.stats["action"])

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface

Dataset loaded: 19631 frames
{'count': array([19631]),
 'max': array([ 72.7734375 , 179.47265625, 164.61914062,  96.59179688,
       123.57421875,  34.9307785 ]),
 'mean': array([ 14.51138199, 146.44867041, 143.31572513,  62.96079529,
        85.83100241,   7.78159506]),
 'min': array([-37.17773438,  48.8671875 ,  40.95703125,   9.66796875,
        56.25      ,   0.        ]),
 'std': array([27.98693199, 34.98953716, 21.46425995, 16.91135693, 12.47836367,
        9.54603304])}


In [19]:
policy_cfg = SmolVLAConfig(
    n_action_steps=10,
    chunk_size=10,
    pretrained_path="lerobot/smolvla_base"
)

print("Building Policy...")
policy = make_policy(
    cfg=policy_cfg,
    ds_meta=dataset.meta
)

Building Policy...
Reducing the number of VLM layers to 16 ...


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
policy.to(device)
print(f"device: {device}")

device: cuda


In [21]:
item = dataset[0]
print(item.keys())
# print(item.items())
batch = {k: v.unsqueeze(0).to(device) for k, v in item.items() if isinstance(v, torch.Tensor)}

print("Batch keys:", batch.keys())
print(f"Image shape: {batch['observation.images.top'].shape}")
print(f"Action shape: {batch['action'].shape}")

c:\Users\19142\.conda\envs\my_robotics\lib\site-packages\torchvision\io\_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(


dict_keys(['observation.images.top', 'observation.images.wrist', 'action', 'observation.state', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index', 'task'])
Batch keys: dict_keys(['observation.images.top', 'observation.images.wrist', 'action', 'observation.state', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index'])
Image shape: torch.Size([1, 3, 480, 640])
Action shape: torch.Size([1, 6])


In [22]:
policy.train()

loss, output_dict = policy.forward(batch)

print(f"Loss: {loss.item()}")
print("Output dict keys:", output_dict.keys())

KeyError: 'observation.language.tokens'